# One to Many

In [1]:
import torch
import numpy as np
import gc

device = 'mps'

In [33]:
class SimpleRNN(torch.nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super().__init__()
        self.I = n_input
        self.H = n_hidden
        self.O = n_output

        self.rnn = torch.nn.RNN(input_size=self.I,
                                hidden_size=self.H,
                                nonlinearity='tanh',
                                batch_first=True)
        
        self.fc = torch.nn.Linear(self.H, self.O)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.H).to(device)

        out, h_0 = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])

        return out.view(-1, 10)
    
class LSTM(torch.nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super().__init__()
        self.I = n_input
        self.H = n_hidden
        self.O = n_output

        self.lstm = torch.nn.LSTM(input_size=self.I,
                                 hidden_size=self.H,
                                 batch_first=True)
        
        self.fc = torch.nn.Linear(self.H, self.O)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.H).to(device)
        c0 = torch.zeros(1, x.size(0), self.H).to(device)

        out, (h_0, c_0) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out.view(-1, 10)
    
class GRU(torch.nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super().__init__()
        self.I = n_input
        self.H = n_hidden
        self.O = n_output

        self.gru = torch.nn.GRU(input_size=self.I,
                                hidden_size=self.H,
                                batch_first=True)
        
        self.fc = torch.nn.Linear(self.H, self.O)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.H).to(device)

        out, h_0= self.gru(x, h0)
        out = self.fc(out[:, -1, :])

        return out.view(-1, 10)

In [34]:
rnn_model = SimpleRNN(n_input=1, n_hidden=40, n_output=10).to(device)
lstm_model = LSTM(n_input=1, n_hidden=40, n_output=10).to(device)
gru_model = GRU(n_input=1, n_hidden=40, n_output=10).to(device)

In [35]:
X = np.random.randint(1, 5, size=(1000, 1, 1)) # 입력 데이터 생성, 1~4 사이의 정수 1000개를 랜덤하게 생성
Y = np.array([[i*j for i in range(1, 11)] for j in X.squeeze()]) # 타겟 데이터 생성, 입력의 배수 10개를 타겟으로 설정

X = torch.from_numpy(X.astype(np.float32)).to(device) # 입력 데이터를 텐서로 변환
Y = torch.from_numpy(Y.astype(np.float32)).to(device) # 타겟 데이터를 텐서로 변환

crietrion = torch.nn.MSELoss()
rnn_optimizer = torch.optim.Adam(rnn_model.parameters())
lstm_optimizer = torch.optim.Adam(lstm_model.parameters())
gru_optimizer = torch.optim.Adam(gru_model.parameters())

In [36]:
for epoch in range(4000):
    outputs = rnn_model(X)
    loss = crietrion(outputs, Y)

    rnn_optimizer.zero_grad()
    loss.backward()
    rnn_optimizer.step()

    if (epoch+1) % 100 == 0: # 100회마다 손실 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

Epoch [100/4000], Loss: 223.3235
Epoch [200/4000], Loss: 128.9948
Epoch [300/4000], Loss: 79.1718
Epoch [400/4000], Loss: 54.5426
Epoch [500/4000], Loss: 41.9000
Epoch [600/4000], Loss: 33.8824
Epoch [700/4000], Loss: 27.0255
Epoch [800/4000], Loss: 20.4305
Epoch [900/4000], Loss: 14.6627
Epoch [1000/4000], Loss: 10.2625
Epoch [1100/4000], Loss: 7.3006
Epoch [1200/4000], Loss: 5.4773
Epoch [1300/4000], Loss: 4.3885
Epoch [1400/4000], Loss: 3.7080
Epoch [1500/4000], Loss: 3.2340
Epoch [1600/4000], Loss: 2.8639
Epoch [1700/4000], Loss: 2.5519
Epoch [1800/4000], Loss: 2.2774
Epoch [1900/4000], Loss: 2.0302
Epoch [2000/4000], Loss: 1.8043
Epoch [2100/4000], Loss: 1.5960
Epoch [2200/4000], Loss: 1.4031
Epoch [2300/4000], Loss: 1.2242
Epoch [2400/4000], Loss: 1.0586
Epoch [2500/4000], Loss: 0.9064
Epoch [2600/4000], Loss: 0.7677
Epoch [2700/4000], Loss: 0.6426
Epoch [2800/4000], Loss: 0.5313
Epoch [2900/4000], Loss: 0.4339
Epoch [3000/4000], Loss: 0.3502
Epoch [3100/4000], Loss: 0.2794
Epoch

In [38]:
# Inference
X_test = torch.tensor([[[2.0]]], dtype=torch.float32).to(device) # 테스트 데이터 생성
print('-' * 20, '추론 결과', '-' * 20)
print(f"Input: 2.0")
output = [round(num, 1) for num in rnn_model(X_test).squeeze().tolist()] # 테스트 데이터에 대한 예측값 계산
answer = list(range(2, 21, 2)) # 정답 리스트 생성

for o, a in zip(output, answer): # 예측값과 정답을 비교하여 출력
    print(f"Output: {o}, 정답: {a}")

-------------------- 추론 결과 --------------------
Input: 2.0
Output: 1.9, 정답: 2
Output: 3.9, 정답: 4
Output: 5.9, 정답: 6
Output: 7.9, 정답: 8
Output: 9.9, 정답: 10
Output: 11.9, 정답: 12
Output: 13.9, 정답: 14
Output: 15.9, 정답: 16
Output: 17.9, 정답: 18
Output: 19.9, 정답: 20


In [39]:
for epoch in range(4000):
    outputs = lstm_model(X)
    loss = crietrion(outputs, Y)

    lstm_optimizer.zero_grad()
    loss.backward()
    lstm_optimizer.step()

    if (epoch+1) % 100 == 0: # 100회마다 손실 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

# Inference
X_test = torch.tensor([[[2.0]]], dtype=torch.float32).to(device) # 테스트 데이터 생성
print('-' * 20, '추론 결과', '-' * 20)
print(f"Input: 2.0")
output = [round(num, 1) for num in lstm_model(X_test).squeeze().tolist()] # 테스트 데이터에 대한 예측값 계산
answer = list(range(2, 21, 2)) # 정답 리스트 생성

for o, a in zip(output, answer): # 예측값과 정답을 비교하여 출력
    print(f"Output: {o}, 정답: {a}")

Epoch [100/4000], Loss: 269.1194
Epoch [200/4000], Loss: 181.0872
Epoch [300/4000], Loss: 108.9135
Epoch [400/4000], Loss: 69.7690
Epoch [500/4000], Loss: 49.6183
Epoch [600/4000], Loss: 38.4204
Epoch [700/4000], Loss: 31.0421
Epoch [800/4000], Loss: 24.5293
Epoch [900/4000], Loss: 18.1234
Epoch [1000/4000], Loss: 12.8872
Epoch [1100/4000], Loss: 9.2720
Epoch [1200/4000], Loss: 6.9650
Epoch [1300/4000], Loss: 5.4439
Epoch [1400/4000], Loss: 4.3448
Epoch [1500/4000], Loss: 3.4903
Epoch [1600/4000], Loss: 2.8007
Epoch [1700/4000], Loss: 2.2375
Epoch [1800/4000], Loss: 1.7787
Epoch [1900/4000], Loss: 1.4080
Epoch [2000/4000], Loss: 1.1116
Epoch [2100/4000], Loss: 0.8764
Epoch [2200/4000], Loss: 0.6909
Epoch [2300/4000], Loss: 0.5449
Epoch [2400/4000], Loss: 0.4299
Epoch [2500/4000], Loss: 0.3390
Epoch [2600/4000], Loss: 0.2669
Epoch [2700/4000], Loss: 0.2093
Epoch [2800/4000], Loss: 0.1630
Epoch [2900/4000], Loss: 0.1258
Epoch [3000/4000], Loss: 0.0961
Epoch [3100/4000], Loss: 0.0724
Epoc

In [40]:
for epoch in range(10000):
    outputs = gru_model(X)
    loss = crietrion(outputs, Y)

    gru_optimizer.zero_grad()
    loss.backward()
    gru_optimizer.step()

    if (epoch+1) % 100 == 0: # 100회마다 손실 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

# Inference
X_test = torch.tensor([[[2.0]]], dtype=torch.float32).to(device) # 테스트 데이터 생성
print('-' * 20, '추론 결과', '-' * 20)
print(f"Input: 2.0")
output = [round(num, 1) for num in gru_model(X_test).squeeze().tolist()] # 테스트 데이터에 대한 예측값 계산
answer = list(range(2, 21, 2)) # 정답 리스트 생성

for o, a in zip(output, answer): # 예측값과 정답을 비교하여 출력
    print(f"Output: {o}, 정답: {a}")

Epoch [100/4000], Loss: 252.6225
Epoch [200/4000], Loss: 155.2488
Epoch [300/4000], Loss: 99.1744
Epoch [400/4000], Loss: 72.4151
Epoch [500/4000], Loss: 56.1284
Epoch [600/4000], Loss: 54.1046
Epoch [700/4000], Loss: 48.2278
Epoch [800/4000], Loss: 43.4265
Epoch [900/4000], Loss: 40.0914
Epoch [1000/4000], Loss: 40.0626
Epoch [1100/4000], Loss: 39.8450
Epoch [1200/4000], Loss: 38.5944
Epoch [1300/4000], Loss: 36.9173
Epoch [1400/4000], Loss: 34.8867
Epoch [1500/4000], Loss: 31.3907
Epoch [1600/4000], Loss: 27.7692
Epoch [1700/4000], Loss: 24.5167
Epoch [1800/4000], Loss: 21.6718
Epoch [1900/4000], Loss: 19.2135
Epoch [2000/4000], Loss: 17.0348
Epoch [2100/4000], Loss: 15.0426
Epoch [2200/4000], Loss: 13.9575
Epoch [2300/4000], Loss: 11.7854
Epoch [2400/4000], Loss: 10.6397
Epoch [2500/4000], Loss: 21.4553
Epoch [2600/4000], Loss: 22.5265
Epoch [2700/4000], Loss: 20.3864
Epoch [2800/4000], Loss: 18.9155
Epoch [2900/4000], Loss: 18.1129
Epoch [3000/4000], Loss: 17.8079
Epoch [3100/4000]